In [1]:
# %load_ext autoreload
# %autoreload 
import torchvision
import torch
import numpy as np
import os
import torch.nn as nn
from models import init_vgg, Mononito
# import art
# import art
# from toolbox.art.attacks.evasion import AdversarialPatch
# import importlib
# importlib.reload(art)
# import art
from art.attacks.evasion import AdversarialPatch
from art.estimators.classification import PyTorchClassifier
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader, RandomSampler
import torchvision.transforms as transforms
from numpy import rollaxis
import matplotlib.pyplot as plt
%matplotlib inline
from torch.nn import Softmax
device ="cuda" if torch.cuda.is_available() else "cpu"
from torch.utils.data.sampler import SubsetRandomSampler
# from tqdm import tqdm


In [2]:
# os.path.exists(TEST_PATH)

In [3]:
TEST_PATH="./data/v2_cam1_cam2_ split_by_driver/Camera 1/train"
WEIGHT_PATH='./Model_weights/'


In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

In [5]:
dataset = torchvision.datasets.ImageFolder(root=TEST_PATH,transform=preprocess)

In [6]:
# inferenceData = RandomSampler(dataset,True,num_samples=32)
test_train_split = .8
val_train_split = .1
shuffle_dataset = True
random_seed= 42
batch_size = 16
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_train_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, test_indices = indices[:split], indices[split:]
train_size = len(train_indices)
validation_split = int(np.floor((1 - val_train_split) * train_size))

train_indices, val_indices = train_indices[:validation_split], train_indices[validation_split:]

In [7]:
# list(inferenceData)
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler, num_workers=4)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler, num_workers=4)

test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         sampler=test_sampler, num_workers=4)

In [8]:
# validData = [dataset[x] for x in list(valid_sampler)]

In [9]:
# dataLoader = DataLoader(dataset,batch_size=32,sampler=inferenceData)
valImages=[]
valLabels=[]

In [10]:
# images, labels = iter(dataLoader).__next__()
for i,(images,labels)in enumerate(validation_loader):
    if i <3:
        valImages.append(images)
        valLabels.append(labels)

In [11]:
images = torch.cat(valImages)
labels = torch.cat(valLabels)

In [12]:
imagesNp = images.numpy()
labelsNp = labels.numpy()
# imagesNp.shape

In [13]:
vggModel = init_vgg()
vggWeights=torch.load(WEIGHT_PATH+'vgg.pt')
vggModel.load_state_dict(vggWeights)
vggModel.eval()
print("Model loaded")

Model loaded


In [14]:
image_shape = (3,224, 224)
clip_values = (0, 255)
nb_classes = 10
batch_size = 16
scale_min = 0.05
scale_max = 0.3
rotation_max = 22.5
learning_rate = 5000.
max_iter = 500
target=0

In [15]:
loss = nn.CrossEntropyLoss()

In [16]:
# PyTorchClassifier?

In [17]:
newModel = PyTorchClassifier(model = vggModel,loss=loss,input_shape=image_shape,
                             nb_classes=nb_classes, channels_first = True,clip_values=clip_values)

In [18]:
ap = AdversarialPatch(classifier=newModel, rotation_max=rotation_max, 
                      scale_min=scale_min, scale_max=scale_max,
                      learning_rate=learning_rate, max_iter=max_iter, batch_size=batch_size)


In [19]:
target_label =np.zeros((labelsNp.shape[0],nb_classes))
target_label[:,target] = 1.
# target_label.shape

In [20]:
patch, patch_mask = ap.generate(x=imagesNp,y=target_label)

Adversarial patch:   3%|▎         | 16/500 [06:14<2:29:51, 18.58s/it]

KeyboardInterrupt: 

In [ ]:
def bgr_to_rgb(x):
    return x[:, :, ::-1]


In [ ]:
patchNew = np.moveaxis(patch,0,-1)

In [ ]:
patchMaskNew = np.moveaxis(patch_mask,0,-1)

In [ ]:
plt.imshow((patchNew * patchMaskNew).astype(np.uint8))

In [ ]:
# tqdm?